# Extract transform load

To get you started you'll need to first run the cell below this to install some python libraries we will be using for this lab. 

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install mysql-connector-python

# Extraction

Now that we have our libraries installed we can focus on reading some files. In the folder I got you to download from moodle there were some files containing data I want you to insert into your database. Some of these files are in csv and others are in JSON format. We will be using a library called pandas to read this data from the disk and load it into our code memory so we can view it before inserting it into our database. 

In [2]:
# import these libraries to help with processing data today
import pandas as pd
import mysql.connector as mysqlc

In [5]:
# this script will load the customer data
customer_data = pd.read_csv('customer_data.csv' )


# this will show the first few lines of that file
customer_data.head()

name         DOB  \
0      Kathy Mccall  1983-12-26   
1  Jennifer Gilbert  1993-04-10   
2     Tonya Stanley  1988-05-24   
3      Manuel Jones  2011-11-08   
4         Sara Lowe  1973-08-05   

                                             address  
0            2657 Karen Track\nTaylormouth, OR 59687  
1                   Unit 4514 Box 0280\nDPO AA 05086  
2  6658 Kimberly Center\nLake Jennifertown, MI 50442  
3  3830 Sandra Lakes Apt. 635\nEast Kevinbury, KS...  
4                         USCGC Palmer\nFPO AP 21633

Having loaded the data it looks like it would be okay for us to load it directly into our customer data table on our database (no need for transformation). To do this we will use the SQL connector library. See below:

In [7]:
# connecting to database using configurations
# may need to adjust password/user/database as appropriate
config = {
  'user': 'root',
  'password': 'password', # you'll need to select a password that works for your mySQL database
  'host': '127.0.0.1',
  'database': 'video_store', # ensure your database has the correct name
  'raise_on_warnings': True
}
connection = mysqlc.connect(**config)
mycursor = connection.cursor()


# constructing an sql statement for the database
sql = "INSERT INTO customer VALUES (%s, %s, %s, %s)"
val = (2, "John", '1990-02-27', "Dublin 12")

#executing the sql statment and committing it to the DB
mycursor.execute(sql, val)
connection.commit()

# print statment to confirm the data was inserted
print(mycursor.rowcount, "record inserted.")
#connection.close()


1 record inserted.


Go to your MySQL shell to check that the record was committed. 

Now we can focus on committing the pandas dataframe (data we loaded) to the database. 

In [18]:
# to get the first row use the following syntax
customer_data.iloc[0]

name                                  Kathy Mccall
DOB                                     1983-12-26
address    2657 Karen Track\nTaylormouth, OR 59687
Name: 0, dtype: object

Now we will iterate through the pandas array and insert the data one row at a time. 

In [8]:
for row in range(customer_data.shape[0]-1):
    row_data = customer_data.iloc[row]
    # store each column in a separate variable
    name = row_data['name']
    DOB = row_data['DOB']
    address = row_data['address']
    
    # constructing an sql statement for the database
    sql = "INSERT INTO customer VALUES (%s, %s, %s, %s)"
    val = (3+row, name, DOB,  address)
    
    try:
        #executing the sql statment and committing it to the DB
        mycursor.execute(sql, val)
    except Exception as e:
        print(e)
    
connection.commit()
# print statment to confirm the data was inserted
print(mycursor.rowcount, "record inserted.")
#connection.close()

1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): Data too long for column 'address' at row 1
1406 (22001): 

Now I want to you complete the same process the for the movies table

In [9]:
#I'll get you started: 
# this script will load the movie data
movie_data = pd.read_csv('movie_data.csv' )


# this will show the first few lines of that file
movie_data.head()

title     release  rating  \
0                 Serious inside else memory if six.  1991-07-26       4   
1     Group through despite cause cause believe son.  1987-07-07       3   
2                   Better available sure form nice.  1979-07-25       4   
3              First policy daughter need kind miss.  1994-04-23       1   
4  Trouble behavior style report size personal pa...  1983-11-03       0   

     genre  
0    scifi  
1  romance  
2    scifi  
3   comedy  
4    scifi

Now we will iterate through the pandas array and insert the data one row at a time. 

Now lets do the same with the transactions for the shop. I've saved these with the same heading as the show database description. However there is one issue. The dates are not in order (take-out date and return data can be in incorrect order). We will need to transform this data. Let's just assume they were inserted in the wrong order. So when one is bigger than the other their values will swap. 

In [49]:
# extract
# this script will load the shop data
shop_data = pd.read_csv('shop_data.csv' )


# this will show the first few lines of that file
shop_data.head()

cust_id  movie_id  cost rent_start_date rent_end_date
0       55        27     3      2015-03-23    2015-02-06
1       19        47     9      2015-03-05    2015-03-15
2       76        34     7      2015-06-27    2015-05-19
3       32        74     3      2015-03-24    2015-02-17
4        7        62     0      2015-01-14    2015-05-23

In [48]:
# tranform and load

# the rest should look something like this. 
import datetime

for row in range(shop_data.shape[0]-1):
    row_data = shop_data.iloc[row]
    # store each column in a separate variable
    cust_id = row_data['cust_id']
    movie_id = row_data['movie_id']
    cost = row_data['cost']
    rent_start = row_data['rent_start_date']
    rent_end = row_data['rent_end_date']
    
    #compare dates by using datetime object
    d1 = datetime.datetime(int(rent_start.split('-')[0]),int( rent_start.split('-')[1]),int( rent_start.split('-')[2]))
    d2 = datetime.datetime(int(rent_end.split('-')[0]),int(rent_end.split('-')[1]),int( rent_end.split('-')[2]))
    
    if(d1>d2):
        temp = rent_start
        rent_start = rent_end
        rent_end = temp
    
    # constructing an sql statement for the database
    sql = "INSERT INTO rental VALUES (%s, %s, %s, %s, %s, %s)"
    val = (str(2+row), rent_start, rent_end, str(cust_id),str(cost),str(movie_id) )
    print(val)
    try:
        #executing the sql statment and committing it to the DB
        mycursor.execute(sql, val)
    except Exception as e:
        print(e)
    
connection.commit()
# print statment to confirm the data was inserted
print(mycursor.rowcount, "record inserted.")
#connection.close()

('2', '2015-02-06', '2015-03-23', '55', '3', '27')
1062 (23000): Duplicate entry '2' for key 'rental.PRIMARY'
('3', '2015-03-05', '2015-03-15', '19', '9', '47')
1062 (23000): Duplicate entry '3' for key 'rental.PRIMARY'
('4', '2015-05-19', '2015-06-27', '76', '7', '34')
1062 (23000): Duplicate entry '4' for key 'rental.PRIMARY'
('5', '2015-02-17', '2015-03-24', '32', '3', '74')
1062 (23000): Duplicate entry '5' for key 'rental.PRIMARY'
('6', '2015-01-14', '2015-05-23', '7', '0', '62')
1062 (23000): Duplicate entry '6' for key 'rental.PRIMARY'
('7', '2015-01-06', '2015-02-19', '41', '0', '54')
1062 (23000): Duplicate entry '7' for key 'rental.PRIMARY'
('8', '2015-05-10', '2015-06-13', '66', '8', '39')
1062 (23000): Duplicate entry '8' for key 'rental.PRIMARY'
('9', '2015-01-06', '2015-06-03', '80', '1', '41')
1062 (23000): Duplicate entry '9' for key 'rental.PRIMARY'
('10', '2015-01-30', '2015-02-27', '42', '3', '32')
1062 (23000): Duplicate entry '10' for key 'rental.PRIMARY'
('11', '2